#To do:
0. Date filter on bureau doesn't make sense as similar filters can't be applied on test data
   Apply own vs other loan providers filters and then aggregate
   Overfitting detection in catboost

1. lgbm find hyperparameters to tune
2. bureau data - Apply date filter to exclude data after loan top-up
3. feval for f1 input as lgbm metric
4. missing data imputation, with business knowledge: city, zip, region etc.
5. Test data has very few missing values (5-10), should we simply drop missing records from train data to reduce error due to imputation

For both catboost and LGBM use objective = F1 (use custom one if possible), metric is meaningless

For catboost:
model = CatBoostClassifier(loss_function=MultiClassObjective())
eval metric vs objective
Total F1(average='macro')

In [131]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

#train_b = pd.read_excel('C:/Users/PALLANA3/Downloads/Data Science/LNTFS 3 AV/Inputs/train_bureau.xlsx')
#test_b=pd.read_excel('C:/Users/PALLANA3/Downloads/Data Science/LNTFS 3 AV/Inputs/test_bureau.xlsx')
test=pd.read_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Inputs/test.csv')
test_id=test.copy()
train=pd.read_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Inputs/train.csv')
train=train.drop('ID',axis=1)
test=test.drop('ID',axis=1)

# income_num = {"10L - 15L":3,"5L - 10L":2,"Less than 5L":1,"More than 15L":4}
# Cred_Cat_num={"Average":2,"Good":3,"Poor":1}
# Gender_num={'Female':0,'Male':1}
# Prod_num={1:1,2:2,'3+':4}
# train=train.replace({"Income": income_num,'Gender':Gender_num,'Credit_Category':Cred_Cat_num,'Product_Holdings':Prod_num})
# test=test.replace({"Income": income_num,'Gender':Gender_num,'Credit_Category':Cred_Cat_num,'Product_Holdings':Prod_num})
# train.Product_Holdings=train.Product_Holdings.astype(int)
# test.Product_Holdings=test.Product_Holdings.astype(int)

Catboost

In [85]:
# #Catboost without gridsearch
    
# from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
# from sklearn.model_selection import KFold,train_test_split
# X,y=train.drop(['Is_Churn'],axis=1),train.Is_Churn

# categorical_features_indices = np.where(X.dtypes =='object')[0]

# err=[]
# y_pred_tot=[]
# #from sklearn.model_selection import KFold,StratifiedKFold
# #fold=KFold(n_splits=5,shuffle=True,random_state=1994)
# #for train_index, test_index in fold.split(X,y):
# #X_train, X_test = X.iloc[train_index], X.iloc[test_index]
# #y_train, y_test = y[train_index], y[test_index]
# #from sklearn.model_selection import train_test_split
# #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)


# print('start')
# cat_model5 = CatBoostClassifier(n_estimators=10, # use large n_estimators deliberately to make use of the early stopping
#                          reg_lambda=5,
#                          eval_metric='TotalF1',
#                          random_seed=13,
#                          learning_rate = 0.3,
#                          depth = 10,
#                          bagging_temperature = 1,cat_features=categorical_features_indices
#                          )
        
# #cat_model5.fit(X_train.values,y_train.values,eval_set=(X_val, y_val),plot=False,use_best_model=True,verbose_eval=50) # early stopping set to 100 to prevent overfitting

# #################  No eval in fit tried    ##################


# cat_model5.fit(X,y,plot=False,use_best_model=True,verbose_eval=100) # early stopping set to 100 to prevent overfitting

# #print('pred...')
#     #p = cat_model5.predict(X_test)
#     #if sum(p<0)>0:
#     #    p=pos(p)
#     #print("err: ",np.sqrt(mean_squared_error(y_test,p)))
#     #err.append(np.sqrt(mean_squared_error(y_test,p)))
# pred_cat = cat_model5.predict(test)
# #f1_score(y_test, pred_c, average='macro')
# #y_pred_tot.append(pred)



In [83]:
#pred_cat = cat_model5.predict(test)

In [84]:
# index=pd.DataFrame({'ID':test_id.ID})
# results=pd.DataFrame(pred_cat.reshape(-1,1)) 
# out=index.merge(results,left_index=True,right_index=True)
# out.columns=['ID','Is_Churn']
# out
# out.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/Catboost_trial_multiple.csv',index=False)

Catboost tuning

In [38]:
from sklearn.metrics import f1_score,make_scorer
f1 = make_scorer(f1_score,average='macro', greater_is_better=True)

In [80]:
#Alternate code for gridsearch with stratkfold\n",
X,y=train.drop(['Is_Churn'],axis=1),train.Is_Churn
params = {'n_estimators': [600,1000],
        'reg_lambda' : [5,9],
        'learning_rate' : [0.01,0.1,0.3],
        'depth' : [5,10],
        'bagging_temperature' : [1]
        }                        
from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV

categorical_features_indices = np.where(X.dtypes =='object')[0]
catb = CatBoostClassifier(cat_features=categorical_features_indices,random_seed=13,early_stopping_rounds=100,
                          eval_metric='TotalF1')

skf = StratifiedKFold(n_splits=4, shuffle = True, random_state = 13)
print('start_gridsearch')
grid = GridSearchCV(estimator=catb, param_grid=params, cv=skf.split(X,y),verbose=100,scoring = f1)
print('start_fit')
grid.fit(X, y)

results = pd.DataFrame(grid.cv_results_)
results.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/tuning Cat3.csv', index=False)

pred_c=[]
pred_c = grid.predict(test)

start_gridsearch
start_fit
Fitting 4 folds for each of 24 candidates, totalling 96 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] bagging_temperature=1, depth=5, learning_rate=0.01, n_estimators=6, reg_lambda=5 
0:	learn: 0.6724995	total: 19ms	remaining: 94.8ms
1:	learn: 0.6712289	total: 41.2ms	remaining: 82.5ms
2:	learn: 0.6685814	total: 47ms	remaining: 47ms
3:	learn: 0.6685814	total: 64.4ms	remaining: 32.2ms
4:	learn: 0.6685814	total: 86.5ms	remaining: 17.3ms
5:	learn: 0.6685814	total: 108ms	remaining: 0us
[CV]  bagging_temperature=1, depth=5, learning_rate=0.01, n_estimators=6, reg_lambda=5, score=0.435, total=   0.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[CV] bagging_temperature=1, depth=5, learning_rate=0.01, n_estimators=6, reg_lambda=5 
0:	learn: 0.6683086	total: 18.2ms	remaining: 91.2ms
1:	learn: 0.6753782	total: 38.6ms	remaining: 77.2ms
2:	learn: 0.6683086	total: 44.9ms	remaining: 44.9ms
3:	l

6:	learn: 0.6685814	total: 173ms	remaining: 74.1ms
7:	learn: 0.6685814	total: 194ms	remaining: 48.5ms
8:	learn: 0.6685814	total: 218ms	remaining: 24.2ms
9:	learn: 0.6685814	total: 231ms	remaining: 0us
[CV]  bagging_temperature=1, depth=5, learning_rate=0.01, n_estimators=10, reg_lambda=9, score=0.435, total=   0.3s
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    3.1s remaining:    0.0s
[CV] bagging_temperature=1, depth=5, learning_rate=0.01, n_estimators=10, reg_lambda=9 
0:	learn: 0.6683086	total: 17.6ms	remaining: 159ms
1:	learn: 0.6753782	total: 39.2ms	remaining: 157ms
2:	learn: 0.6683086	total: 47.9ms	remaining: 112ms
3:	learn: 0.6683086	total: 67.4ms	remaining: 101ms
4:	learn: 0.6683086	total: 88.9ms	remaining: 88.9ms
5:	learn: 0.6683086	total: 111ms	remaining: 73.9ms
6:	learn: 0.6683086	total: 132ms	remaining: 56.4ms
7:	learn: 0.6683086	total: 154ms	remaining: 38.5ms
8:	learn: 0.6683086	total: 175ms	remaining: 19.5ms
9:	learn: 0.6683086	total: 188ms	remaining: 0us
[CV]  b

0:	learn: 0.6683086	total: 21ms	remaining: 189ms
1:	learn: 0.6753782	total: 44.6ms	remaining: 178ms
2:	learn: 0.6683086	total: 84.4ms	remaining: 197ms
3:	learn: 0.6683086	total: 104ms	remaining: 156ms
4:	learn: 0.6683086	total: 128ms	remaining: 128ms
5:	learn: 0.6683086	total: 152ms	remaining: 101ms
6:	learn: 0.6683086	total: 173ms	remaining: 73.9ms
7:	learn: 0.6692608	total: 195ms	remaining: 48.9ms
8:	learn: 0.6687850	total: 216ms	remaining: 24.1ms
9:	learn: 0.6683086	total: 233ms	remaining: 0us
[CV]  bagging_temperature=1, depth=5, learning_rate=0.1, n_estimators=10, reg_lambda=5, score=0.435, total=   0.3s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:    6.1s remaining:    0.0s
[CV] bagging_temperature=1, depth=5, learning_rate=0.1, n_estimators=10, reg_lambda=5 
0:	learn: 0.6683716	total: 35ms	remaining: 315ms
1:	learn: 0.6683716	total: 63.3ms	remaining: 253ms
2:	learn: 0.6683716	total: 114ms	remaining: 267ms
3:	learn: 0.6683716	total: 156ms	remaining: 234ms
4:	learn: 0.6683

3:	learn: 0.6738321	total: 85.2ms	remaining: 42.6ms
4:	learn: 0.6749705	total: 108ms	remaining: 21.7ms
5:	learn: 0.6754357	total: 134ms	remaining: 0us
[CV]  bagging_temperature=1, depth=5, learning_rate=0.3, n_estimators=6, reg_lambda=9, score=0.445, total=   0.2s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:    9.9s remaining:    0.0s
[CV] bagging_temperature=1, depth=5, learning_rate=0.3, n_estimators=6, reg_lambda=9 
0:	learn: 0.6683716	total: 16ms	remaining: 80ms
1:	learn: 0.6683716	total: 35ms	remaining: 70ms
2:	learn: 0.6683716	total: 57.3ms	remaining: 57.3ms
3:	learn: 0.6683716	total: 80ms	remaining: 40ms
4:	learn: 0.6683716	total: 102ms	remaining: 20.4ms
5:	learn: 0.6683716	total: 123ms	remaining: 0us
[CV]  bagging_temperature=1, depth=5, learning_rate=0.3, n_estimators=6, reg_lambda=9, score=0.435, total=   0.2s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:   10.1s remaining:    0.0s
[CV] bagging_temperature=1, depth=5, learning_rate=0.3, n_estimators=6, reg_lamb

5:	learn: 0.6683086	total: 203ms	remaining: 0us
[CV]  bagging_temperature=1, depth=10, learning_rate=0.01, n_estimators=6, reg_lambda=5, score=0.435, total=   0.3s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   13.0s remaining:    0.0s
[CV] bagging_temperature=1, depth=10, learning_rate=0.01, n_estimators=6, reg_lambda=5 
0:	learn: 0.6683716	total: 13ms	remaining: 64.9ms
1:	learn: 0.6683716	total: 39.5ms	remaining: 79ms
2:	learn: 0.6683716	total: 64ms	remaining: 64ms
3:	learn: 0.6683716	total: 76.7ms	remaining: 38.3ms
4:	learn: 0.6683716	total: 82ms	remaining: 16.4ms
5:	learn: 0.6683716	total: 131ms	remaining: 0us
[CV]  bagging_temperature=1, depth=10, learning_rate=0.01, n_estimators=6, reg_lambda=5, score=0.435, total=   0.2s
[Parallel(n_jobs=1)]: Done  51 out of  51 | elapsed:   13.2s remaining:    0.0s
[CV] bagging_temperature=1, depth=10, learning_rate=0.01, n_estimators=6, reg_lambda=5 
0:	learn: 0.6683716	total: 14.1ms	remaining: 70.5ms
1:	learn: 0.6683716	total: 42.1ms	

4:	learn: 0.6683716	total: 85ms	remaining: 85ms
5:	learn: 0.6683716	total: 134ms	remaining: 89.5ms
6:	learn: 0.6683716	total: 184ms	remaining: 78.8ms
7:	learn: 0.6683716	total: 238ms	remaining: 59.5ms
8:	learn: 0.6683716	total: 285ms	remaining: 31.6ms
9:	learn: 0.6683716	total: 337ms	remaining: 0us
[CV]  bagging_temperature=1, depth=10, learning_rate=0.01, n_estimators=10, reg_lambda=9, score=0.435, total=   0.4s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:   17.6s remaining:    0.0s
[CV] bagging_temperature=1, depth=10, learning_rate=0.01, n_estimators=10, reg_lambda=9 
0:	learn: 0.6683716	total: 14.3ms	remaining: 128ms
1:	learn: 0.6683716	total: 44.3ms	remaining: 177ms
2:	learn: 0.6683716	total: 73.8ms	remaining: 172ms
3:	learn: 0.6683716	total: 91.1ms	remaining: 137ms
4:	learn: 0.6683716	total: 145ms	remaining: 145ms
5:	learn: 0.6683716	total: 193ms	remaining: 128ms
6:	learn: 0.6683716	total: 245ms	remaining: 105ms
7:	learn: 0.6683716	total: 286ms	remaining: 71.4ms
8:	learn:

1:	learn: 0.6683716	total: 71.9ms	remaining: 288ms
2:	learn: 0.6683716	total: 84.8ms	remaining: 198ms
3:	learn: 0.6683716	total: 135ms	remaining: 202ms
4:	learn: 0.6688480	total: 171ms	remaining: 171ms
5:	learn: 0.6683716	total: 196ms	remaining: 131ms
6:	learn: 0.6688480	total: 247ms	remaining: 106ms
7:	learn: 0.6693237	total: 305ms	remaining: 76.2ms
8:	learn: 0.6716921	total: 356ms	remaining: 39.6ms
9:	learn: 0.6683716	total: 378ms	remaining: 0us
[CV]  bagging_temperature=1, depth=10, learning_rate=0.1, n_estimators=10, reg_lambda=5, score=0.435, total=   0.4s
[Parallel(n_jobs=1)]: Done  76 out of  76 | elapsed:   21.8s remaining:    0.0s
[CV] bagging_temperature=1, depth=10, learning_rate=0.1, n_estimators=10, reg_lambda=9 
0:	learn: 0.6932303	total: 50.6ms	remaining: 455ms
1:	learn: 0.6685814	total: 56.2ms	remaining: 225ms
2:	learn: 0.6685814	total: 75.1ms	remaining: 175ms
3:	learn: 0.6685814	total: 96.7ms	remaining: 145ms
4:	learn: 0.6685814	total: 154ms	remaining: 154ms
5:	learn: 

0:	learn: 0.6993183	total: 48.8ms	remaining: 439ms
1:	learn: 0.6685814	total: 54.2ms	remaining: 217ms
2:	learn: 0.6689590	total: 102ms	remaining: 239ms
3:	learn: 0.6684829	total: 156ms	remaining: 233ms
4:	learn: 0.6713296	total: 183ms	remaining: 183ms
5:	learn: 0.6788045	total: 235ms	remaining: 157ms
6:	learn: 0.6718017	total: 250ms	remaining: 107ms
7:	learn: 0.6760210	total: 301ms	remaining: 75.3ms
8:	learn: 0.6845367	total: 352ms	remaining: 39.1ms
9:	learn: 0.6908219	total: 405ms	remaining: 0us
[CV]  bagging_temperature=1, depth=10, learning_rate=0.3, n_estimators=10, reg_lambda=5, score=0.448, total=   0.5s
[Parallel(n_jobs=1)]: Done  89 out of  89 | elapsed:   26.9s remaining:    0.0s
[CV] bagging_temperature=1, depth=10, learning_rate=0.3, n_estimators=10, reg_lambda=5 
0:	learn: 0.6859221	total: 49.1ms	remaining: 442ms
1:	learn: 0.6828317	total: 99.3ms	remaining: 397ms
2:	learn: 0.6683086	total: 117ms	remaining: 274ms
3:	learn: 0.6711572	total: 169ms	remaining: 253ms
4:	learn: 0.

In [81]:
index=pd.DataFrame({'ID':test_id.ID})
results=pd.DataFrame(pred_c.reshape(-1,1)) 
out=index.merge(results,left_index=True,right_index=True)
out.columns=['ID','Is_Churn']
out
out.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/Catboost4.csv',index=False)

LGBM

In [132]:
from sklearn.metrics import f1_score,make_scorer
f1 = make_scorer(f1_score,average='macro', greater_is_better=True)

In [133]:
param_test ={
             'subsample': [0.6, 0.8, 1.0],
             'colsample_bytree': [0.4,0.6, 0.8,1],
             'max_depth': [ 6,9,12,20],
             'learning_rate':[0.1,0.01,0.3],
             'n_estimators':[40,70,100,250],
             'num_leaves':[50,100,250,400],
             'reg_lambda': [1,0.2,0.7,0.5]
            }

In [ ]:
from lightgbm import LGBMClassifier

X,y=train.drop(['Is_Churn'],axis=1),train.Is_Churn
objects=X.select_dtypes('object').columns.to_list()
for col in objects:
            X[col] = X[col].astype('category')
            test[col] = test[col].astype('category')

#categorical_features_indices = np.where(X.dtypes =='object')[0]


lgbm = LGBMClassifier(categorical_features = categorical_features_indices,random_state=13,
                      metric='f1',is_unbalance = True)

#metric=f1,

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=lgbm, param_grid=param_test,scoring='f1',n_jobs=-1, cv=skf.split(Xd,yd),verbose=100 )
# n_jobs=4,
grid.fit(X, y)

print('Best score: {} params: {} '.format(grid.best_score_, grid.best_params_))

results = pd.DataFrame(grid.cv_results_)
results.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/LGBM_tuning3.csv',index=False)

pred_l = grid.predict(test)


Fitting 3 folds for each of 9216 candidates, totalling 27648 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1996s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  13

[Parallel(n_jobs=-1)]: Done 244 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 246 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 248 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 250 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 254 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 258 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 260 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 262 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 266 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 270 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 274 tasks      | elapsed:    9.2s
[Paralle

[Parallel(n_jobs=-1)]: Done 495 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 497 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 498 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done 499 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done 500 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done 501 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 502 tasks      | elapsed:   25.0s
[Parallel(n_jobs=-1)]: Done 503 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 505 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done 506 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done 507 tasks      | elapsed:   25.6s
[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed:   25.6s
[Parallel(n_jobs=-1)]: Done 509 tasks      | elapsed:   25.8s
[Parallel(n_jobs=-1)]: Done 510 tasks      | elapsed:   26.0s
[Paralle

[Parallel(n_jobs=-1)]: Done 629 tasks      | elapsed:   35.9s
[Parallel(n_jobs=-1)]: Done 630 tasks      | elapsed:   35.9s
[Parallel(n_jobs=-1)]: Done 631 tasks      | elapsed:   35.9s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   35.9s
[Parallel(n_jobs=-1)]: Done 633 tasks      | elapsed:   35.9s
[Parallel(n_jobs=-1)]: Done 634 tasks      | elapsed:   36.0s
[Parallel(n_jobs=-1)]: Done 635 tasks      | elapsed:   36.1s
[Parallel(n_jobs=-1)]: Done 636 tasks      | elapsed:   36.2s
[Parallel(n_jobs=-1)]: Done 637 tasks      | elapsed:   36.2s
[Parallel(n_jobs=-1)]: Done 638 tasks      | elapsed:   36.2s
[Parallel(n_jobs=-1)]: Done 639 tasks      | elapsed:   36.2s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:   36.2s
[Parallel(n_jobs=-1)]: Done 641 tasks      | elapsed:   36.3s
[Parallel(n_jobs=-1)]: Done 642 tasks      | elapsed:   36.3s
[Parallel(n_jobs=-1)]: Done 643 tasks      | elapsed:   36.4s
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:   36.4s
[Paralle

[Parallel(n_jobs=-1)]: Done 763 tasks      | elapsed:   41.6s
[Parallel(n_jobs=-1)]: Done 764 tasks      | elapsed:   41.7s
[Parallel(n_jobs=-1)]: Done 765 tasks      | elapsed:   41.8s
[Parallel(n_jobs=-1)]: Done 766 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 767 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 769 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 770 tasks      | elapsed:   42.0s
[Parallel(n_jobs=-1)]: Done 771 tasks      | elapsed:   42.1s
[Parallel(n_jobs=-1)]: Done 772 tasks      | elapsed:   42.2s
[Parallel(n_jobs=-1)]: Done 773 tasks      | elapsed:   42.2s
[Parallel(n_jobs=-1)]: Done 774 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-1)]: Done 775 tasks      | elapsed:   42.4s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   42.5s
[Parallel(n_jobs=-1)]: Done 777 tasks      | elapsed:   42.5s
[Parallel(n_jobs=-1)]: Done 778 tasks      | elapsed:   42.6s
[Paralle

...

[Parallel(n_jobs=-1)]: Done 2749 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2750 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2751 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2752 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2753 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2754 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2755 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2756 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2757 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2758 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2759 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2760 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 2761 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 2762 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 2763 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 2764 tasks      | elapsed: 

[Parallel(n_jobs=-1)]: Done 2882 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2883 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2884 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2885 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2886 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2887 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2888 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2889 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2890 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2891 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2892 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2893 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2894 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2895 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2896 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2897 tasks      | elapsed: 

[Parallel(n_jobs=-1)]: Done 3015 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3016 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3017 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3018 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3019 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3020 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3021 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3022 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3023 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3024 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3025 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3026 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3027 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3028 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3029 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3030 tasks      | elapsed: 

[Parallel(n_jobs=-1)]: Done 3146 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3147 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3148 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3149 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3150 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3151 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3152 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3153 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3154 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3155 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3156 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3157 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3158 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3159 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3161 tasks      | elapsed: 

[Parallel(n_jobs=-1)]: Done 3277 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3278 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3279 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3280 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3281 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3282 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3283 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3284 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3285 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3286 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3287 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3288 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3289 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3290 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3291 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3292 tasks      | elapsed: 

In [ ]:
index=pd.DataFrame({'ID':test_id.ID})
results=pd.DataFrame(pred_l.reshape(-1,1)) 
out=index.merge(results,left_index=True,right_index=True)
out.columns=['ID','Is_Churn']
out
out.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/Lgbm3.csv',index=False)

XGB

In [113]:
Obj=train.select_dtypes('object').columns.to_list()
train = pd.get_dummies(train, columns=Obj,drop_first=True)
test = pd.get_dummies(test, columns=Obj,drop_first=True)

In [115]:
from sklearn.metrics import f1_score,make_scorer
f1 = make_scorer(f1_score,average='macro', greater_is_better=True)

In [118]:
#Alternate code for gridsearch/tuning

# A parameter grid for XGBoost
params = {
        'subsample': [0.4,0.8,1],
        'colsample_bytree': [0.5,0.7,0.9],
        'max_depth': [ 5, 9,15,25],
        'learning_rate':[0.01,0.1,0.3],
        'n_estimators':[40,80,120,200]
        }



Xd,yd=train.drop(['Is_Churn'],axis=1),train.Is_Churn
#from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.metrics import f1_score
xg = xgb.XGBClassifier(eval_metric=f1_score,nthread=-1,verbose=100)          

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=xg, param_grid=params,scoring='f1',n_jobs=-1, cv=skf.split(Xd,yd),verbose=100 )
# n_jobs=4,
grid.fit(Xd, yd)

results = pd.DataFrame(grid.cv_results_)
results.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/XGB_tuning1.csv',index=False)

pred_x = grid.predict(test)
#y_test = grid.best_estimator_.predict_proba(test)
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
#results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 

[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:   51.1s
[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done 134 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:   52.7s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:   54.1s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:   54.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   55.2s
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:   55.8s
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:   56.0s
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:   56.3s
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:   56.4s
[Parallel(n_jobs=-1)]: Done 143 tasks      | elapsed:   56.7s
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:   56.9s
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:   57.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   57.3s
[Parallel(n_jobs=-1)]: Done 147 tasks      | elapsed:   57.4s
[Paralle

[Parallel(n_jobs=-1)]: Done 265 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 266 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 267 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 269 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 270 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 271 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 273 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 274 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 275 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 279 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  1.8min
[Paralle

[Parallel(n_jobs=-1)]: Done 398 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 399 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 401 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 403 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 404 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 405 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 406 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 407 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 408 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 409 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 411 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 412 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 413 tasks      | elapsed:  2.5min
[Paralle

[Parallel(n_jobs=-1)]: Done 531 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 532 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 533 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 534 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 535 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 536 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 537 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 538 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 539 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 540 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 541 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 542 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 543 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 544 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 545 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 546 tasks      | elapsed:  3.6min
[Paralle

...

[Parallel(n_jobs=-1)]: Done 800 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 801 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 802 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 803 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 804 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 805 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 806 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 807 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 808 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 809 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 810 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 811 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 812 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 813 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 814 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 815 tasks      | elapsed:  5.8min
[Paralle

[Parallel(n_jobs=-1)]: Done 934 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 935 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 936 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 937 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 938 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 939 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 940 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 941 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 942 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 943 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 944 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 945 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 946 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 947 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 948 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 949 tasks      | elapsed:  6.8min
[Paralle

[Parallel(n_jobs=-1)]: Done 1066 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 1067 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 1068 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 1069 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 1070 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 1071 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 1072 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1073 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1074 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1075 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1076 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1077 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1078 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1079 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1080 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1081 tasks      | elapsed: 

[Parallel(n_jobs=-1)]: Done 1198 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1199 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1200 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1201 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1203 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1204 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1205 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1206 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1207 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1208 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1209 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1210 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1211 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1212 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1213 tasks      | elapsed: 

C:\Users\PALLANA3\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [119]:
index=pd.DataFrame({'ID':test_id.ID})
results=pd.DataFrame(pred_x.reshape(-1,1)) 
out=index.merge(results,left_index=True,right_index=True)
out.columns=['ID','Is_Churn']
out
out.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/XGB2.csv',index=False)

In [116]:
# #XGBoost code,using tuned models to predict

# X,y=train.drop(['Is_Churn'],axis=1),train.Is_Churn

# from sklearn.model_selection import StratifiedKFold,train_test_split
# from sklearn.metrics import f1_score

# import xgboost as xgb
# xg_reg = xgb.XGBClassifier(eval_metric=f1_score,learning_rate = 0.3,
#                 max_depth = 9,colsample_bytree=.9, subsample=0.4, n_estimators = 80,verbose=50)


# from sklearn.model_selection import KFold,StratifiedKFold
# #fold=StratifiedKFold(n_splits=5,shuffle=True,random_state=1994)

# #for train_index, test_index in fold.split(X,y):
# #    X_train, X_test = Xd.iloc[train_index], Xd.iloc[test_index]
# #    y_train, y_test = yd[train_index], yd[test_index]
# print('start')
# xg_reg.fit(X,y)
# #    print('pred...')
# #    p=xg_reg.predict(X_test)
# #    print("err: ",100*np.sqrt(mean_squared_error(y_test,p)))
# #    err.append(100*np.sqrt(mean_squared_error(y_test,p)))
# pred_x = xg_reg.predict(test)

Random Forest

In [57]:
Obj=train.select_dtypes('object').columns.to_list()
train = pd.get_dummies(train, columns=Obj,drop_first=True)
test = pd.get_dummies(test, columns=Obj,drop_first=True)

In [58]:
from sklearn.metrics import f1_score,make_scorer
f1 = make_scorer(f1_score,average='macro', greater_is_better=True)

In [64]:
#Alternate code for gridsearch/tuning

params = {
        'max_depth': [20,30,50],
        'max_features': [0.5,0.7,1],
        'min_samples_leaf': [25,50,100],
        'min_samples_split': [100,200,400,600],
        'n_estimators': [250,400, 900]
         }

from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

err=[]
y_pred_tot=[]

Xd,yd=train.drop(['Is_Churn'],axis=1),train.Is_Churn
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(verbose=100)
#criterion=f1_score,
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=rf, param_grid=params,scoring='f1', cv=skf.split(Xd,yd),n_jobs=-1,verbose=100 )
# n_jobs=4,
grid.fit(Xd, yd)
#print('\n All results:')
#print(grid.cv_results_)
#print('\n Best estimator:')
#print(grid.best_estimator_)
#print('\n Best score:')
#print(grid.best_score_ * 2 - 1)
#print('\n Best parameters:')
#print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
 
results.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/RF_Tuning2.csv',index=False)
#y_test = grid.best_estimator_.predict_proba(test)
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
#results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

Fitting 3 folds for each of 324 candidates, totalling 972 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   17.2s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  

[Parallel(n_jobs=-1)]: Done 131 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 134 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 143 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.8min
[Paralle

[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 265 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 266 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 267 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 269 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 270 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 271 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 273 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 274 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 275 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 279 tasks      | elapsed:  3.1min
[Paralle

[Parallel(n_jobs=-1)]: Done 397 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 398 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 399 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 401 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 403 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 404 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 405 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 406 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 407 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 408 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 409 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 411 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 412 tasks      | elapsed:  4.4min
[Paralle

[Parallel(n_jobs=-1)]: Done 530 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 531 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 532 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 533 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 534 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 535 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 536 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 537 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 538 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 539 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 540 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 541 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 542 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 543 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 544 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 545 tasks      | elapsed:  5.9min
[Paralle

...

[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 797 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 798 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 799 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 800 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 801 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 802 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 803 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 804 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 805 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 806 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 807 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 808 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 809 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 810 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 811 tasks      | elapsed:  8.4min
[Paralle

[Parallel(n_jobs=-1)]: Done 929 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 930 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 931 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 932 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 933 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 934 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 935 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 936 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 937 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 938 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 939 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 940 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 941 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 942 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 943 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 944 tasks      | elapsed:  9.7min
[Paralle

building tree 64 of 900
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    1.1s remaining:    0.0s
building tree 65 of 900
[Parallel(n_jobs=1)]: Done  65 out of  65 | elapsed:    1.1s remaining:    0.0s
building tree 66 of 900
[Parallel(n_jobs=1)]: Done  66 out of  66 | elapsed:    1.1s remaining:    0.0s
building tree 67 of 900
[Parallel(n_jobs=1)]: Done  67 out of  67 | elapsed:    1.1s remaining:    0.0s
building tree 68 of 900
[Parallel(n_jobs=1)]: Done  68 out of  68 | elapsed:    1.2s remaining:    0.0s
building tree 69 of 900
[Parallel(n_jobs=1)]: Done  69 out of  69 | elapsed:    1.2s remaining:    0.0s
building tree 70 of 900
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    1.2s remaining:    0.0s
building tree 71 of 900
[Parallel(n_jobs=1)]: Done  71 out of  71 | elapsed:    1.2s remaining:    0.0s
building tree 72 of 900
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:    1.2s remaining:    0.0s
building tree 73 of 900
[Parallel(n_jobs=1)]: Done  73 out of  7

building tree 278 of 900
building tree 279 of 900
building tree 280 of 900
building tree 281 of 900
building tree 282 of 900
building tree 283 of 900
building tree 284 of 900
building tree 285 of 900
building tree 286 of 900
building tree 287 of 900
building tree 288 of 900
building tree 289 of 900
building tree 290 of 900
building tree 291 of 900
building tree 292 of 900
building tree 293 of 900
building tree 294 of 900
building tree 295 of 900
building tree 296 of 900
building tree 297 of 900
building tree 298 of 900
building tree 299 of 900
building tree 300 of 900
building tree 301 of 900
building tree 302 of 900
building tree 303 of 900
building tree 304 of 900
building tree 305 of 900
building tree 306 of 900
building tree 307 of 900
building tree 308 of 900
building tree 309 of 900
building tree 310 of 900
building tree 311 of 900
building tree 312 of 900
building tree 313 of 900
building tree 314 of 900
building tree 315 of 900
building tree 316 of 900
building tree 317 of 900


building tree 615 of 900
building tree 616 of 900
building tree 617 of 900
building tree 618 of 900
building tree 619 of 900
building tree 620 of 900
building tree 621 of 900
building tree 622 of 900
building tree 623 of 900
building tree 624 of 900
building tree 625 of 900
building tree 626 of 900
building tree 627 of 900
building tree 628 of 900
building tree 629 of 900
building tree 630 of 900
building tree 631 of 900
building tree 632 of 900
building tree 633 of 900
building tree 634 of 900
building tree 635 of 900
building tree 636 of 900
building tree 637 of 900
building tree 638 of 900
building tree 639 of 900
building tree 640 of 900
building tree 641 of 900
building tree 642 of 900
building tree 643 of 900
building tree 644 of 900
building tree 645 of 900
building tree 646 of 900
building tree 647 of 900
building tree 648 of 900
building tree 649 of 900
building tree 650 of 900
building tree 651 of 900
building tree 652 of 900
building tree 653 of 900
building tree 654 of 900


In [65]:
Xd,yd=train.drop(['Is_Churn'],axis=1),train.Is_Churn

from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier

err=[]
y_pred_tot=[]
#from sklearn.model_selection import KFold,StratifiedKFold
#fold=StratifiedKFold(n_splits=5,shuffle=True,random_state=1994)

#for train_index, test_index in fold.split(Xd,yd):
#    X_train, X_test = Xd.iloc[train_index], Xd.iloc[test_index]
#    y_train, y_test = yd[train_index], yd[test_index]
#    print('start')
rf=RandomForestClassifier(n_estimators=900,max_depth=50,max_features=0.7,min_samples_leaf=25,min_samples_split=100,verbose=100,random_state=13,n_jobs = -1)
rf.fit(Xd,yd)
#    print('pred...')
#    p=xg_reg.predict(X_test)
#    print("err: ",100*np.sqrt(mean_squared_error(y_test,p)))
#    err.append(100*np.sqrt(mean_squared_error(y_test,p)))
pred_r = rf.predict(test)
#y_pred_tot.append(pred)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
building tree 1 of 900
building tree 2 of 900
building tree 3 of 900
building tree 4 of 900
building tree 5 of 900
building tree 6 of 900building tree 7 of 900

building tree 8 of 900
building tree 9 of 900
building tree 10 of 900
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
building tree 11 of 900[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.0s

building tree 12 of 900[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.0s

building tree 13 of 900[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s
building tree 14 of 900
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s

building tree 15 of 900[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.0s

building tree 16 of 900
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.0s
building tree 17 of 900
[Parallel(n_jobs=-1)]: D


building tree 106 of 900building tree 107 of 900building tree 108 of 900
building tree 109 of 900


[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  99 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 101 tasks      | elapsed:    0.3s
building tree 110 of 900[Parallel(n_jobs=-1)]: Done 102 tasks      | elapsed:    0.3s

building tree 111 of 900[Parallel(n_jobs=-1)]: Done 103 tasks      | elapsed:    0.3s

building tree 112 of 900[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    0.3s

building tree 113 of 900
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:    0.3s
building tree 114 of 900building tree 115 of 900[Parallel(n_jobs=-1)]: Done 106 tasks      | elapsed:    0.3s

[Parallel(n_jobs=-1)]: Done 107 tasks      | elapsed:    0.3s

building tree 116 of 900building tree 117 of 900[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed:    0.3s

[Parallel(n_jobs=-


building tree 235 of 900[Parallel(n_jobs=-1)]: Done 227 tasks      | elapsed:    0.8s

building tree 236 of 900
[Parallel(n_jobs=-1)]: Done 228 tasks      | elapsed:    0.8s
building tree 237 of 900[Parallel(n_jobs=-1)]: Done 229 tasks      | elapsed:    0.8s

building tree 238 of 900
[Parallel(n_jobs=-1)]: Done 230 tasks      | elapsed:    0.8s
building tree 239 of 900building tree 240 of 900[Parallel(n_jobs=-1)]: Done 231 tasks      | elapsed:    0.9s
building tree 241 of 900


building tree 242 of 900
[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 234 tasks      | elapsed:    0.9s
building tree 243 of 900[Parallel(n_jobs=-1)]: Done 235 tasks      | elapsed:    0.9s

building tree 244 of 900[Parallel(n_jobs=-1)]: Done 236 tasks      | elapsed:    0.9s
building tree 245 of 900

[Parallel(n_jobs=-1)]: Done 237 tasks      | elapsed:    0.9s
building tree 246 of 900building tree 247 

building tree 375 of 900
[Parallel(n_jobs=-1)]: Done 367 tasks      | elapsed:    1.4s
building tree 376 of 900building tree 377 of 900[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 369 tasks      | elapsed:    1.4s


building tree 378 of 900[Parallel(n_jobs=-1)]: Done 370 tasks      | elapsed:    1.4s

building tree 379 of 900
[Parallel(n_jobs=-1)]: Done 371 tasks      | elapsed:    1.4s
building tree 380 of 900[Parallel(n_jobs=-1)]: Done 372 tasks      | elapsed:    1.4s

building tree 381 of 900[Parallel(n_jobs=-1)]: Done 373 tasks      | elapsed:    1.4s

building tree 382 of 900[Parallel(n_jobs=-1)]: Done 374 tasks      | elapsed:    1.4s

building tree 383 of 900
[Parallel(n_jobs=-1)]: Done 375 tasks      | elapsed:    1.4s
building tree 384 of 900[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:    1.4s

building tree 385 of 900[Parallel(n_jobs=-1)]: Done 377 tasks      | elapsed:    1.4s

building tree 386 of 900[Parallel(n_jobs=-1

building tree 484 of 900[Parallel(n_jobs=-1)]: Done 476 tasks      | elapsed:    1.8s

building tree 485 of 900[Parallel(n_jobs=-1)]: Done 477 tasks      | elapsed:    1.8s

building tree 486 of 900building tree 487 of 900

[Parallel(n_jobs=-1)]: Done 478 tasks      | elapsed:    1.8s
building tree 488 of 900[Parallel(n_jobs=-1)]: Done 479 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 480 tasks      | elapsed:    1.8s

building tree 489 of 900[Parallel(n_jobs=-1)]: Done 481 tasks      | elapsed:    1.8s
building tree 490 of 900
[Parallel(n_jobs=-1)]: Done 482 tasks      | elapsed:    1.8s

building tree 491 of 900[Parallel(n_jobs=-1)]: Done 483 tasks      | elapsed:    1.8s

building tree 492 of 900building tree 493 of 900
[Parallel(n_jobs=-1)]: Done 484 tasks      | elapsed:    1.8s

[Parallel(n_jobs=-1)]: Done 485 tasks      | elapsed:    1.8s
building tree 494 of 900[Parallel(n_jobs=-1)]: Done 486 tasks      | elapsed:    1.8s
building tree 495 of 900
[Parallel(n_jobs=-1

...


building tree 703 of 900building tree 704 of 900
building tree 705 of 900[Parallel(n_jobs=-1)]: Done 695 tasks      | elapsed:    2.5s

[Parallel(n_jobs=-1)]: Done 696 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 697 tasks      | elapsed:    2.5s

building tree 706 of 900building tree 707 of 900building tree 708 of 900[Parallel(n_jobs=-1)]: Done 698 tasks      | elapsed:    2.5s



[Parallel(n_jobs=-1)]: Done 699 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 700 tasks      | elapsed:    2.5s
building tree 709 of 900[Parallel(n_jobs=-1)]: Done 701 tasks      | elapsed:    2.5s

building tree 710 of 900
[Parallel(n_jobs=-1)]: Done 702 tasks      | elapsed:    2.6s
building tree 711 of 900[Parallel(n_jobs=-1)]: Done 703 tasks      | elapsed:    2.6s
building tree 712 of 900
building tree 713 of 900
building tree 714 of 900[Parallel(n_jobs=-1)]: Done 704 tasks      | elapsed:    2.6s


[Parallel(n_jobs=-1)]: Done 705 tasks      | elapsed:    2.6s
[Parallel(n_jobs=


building tree 815 of 900building tree 816 of 900
[Parallel(n_jobs=-1)]: Done 807 tasks      | elapsed:    2.9s
building tree 817 of 900building tree 818 of 900building tree 819 of 900

building tree 820 of 900[Parallel(n_jobs=-1)]: Done 808 tasks      | elapsed:    2.9s


[Parallel(n_jobs=-1)]: Done 809 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 810 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 811 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 812 tasks      | elapsed:    2.9s
building tree 821 of 900[Parallel(n_jobs=-1)]: Done 813 tasks      | elapsed:    2.9s


building tree 822 of 900[Parallel(n_jobs=-1)]: Done 814 tasks      | elapsed:    2.9s
building tree 823 of 900[Parallel(n_jobs=-1)]: Done 815 tasks      | elapsed:    2.9s


building tree 824 of 900building tree 825 of 900building tree 826 of 900

building tree 827 of 900
building tree 828 of 900building tree 829 of 900

[Parallel(n_jobs=-1)]: Done 816 tasks      | elapsed:    2.9s

[Par

[Parallel(n_jobs=8)]: Done  89 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  90 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  91 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  92 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  93 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  94 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  95 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  96 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  98 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  99 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 101 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 102 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 103 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Do

[Parallel(n_jobs=8)]: Done 483 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 484 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 485 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 486 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 487 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 488 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 489 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 490 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 491 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 492 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 493 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 494 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 495 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 496 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 497 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 498 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Do

[Parallel(n_jobs=8)]: Done 900 out of 900 | elapsed:    0.3s finished


In [66]:
index=pd.DataFrame({'ID':test_id.ID})
results=pd.DataFrame(pred_r.reshape(-1,1)) 
out=index.merge(results,left_index=True,right_index=True)
out.columns=['ID','Is_Churn']
out
out.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/RF2.csv',index=False)

In [18]:
############# Plot Feature importance ##################

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(lgbm.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()
#plt.savefig('lgbm_importances-01.png')

NotFittedError: No feature_importances found. Need to call fit beforehand.

Ensemble

In [19]:
index=pd.DataFrame({'ID':test_id.ID})
results=pd.DataFrame(pred_x.reshape(-1,1)) 
out=index.merge(results,left_index=True,right_index=True)
out.columns=['ID','Is_Churn']
out
out.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/XGB1.csv',index=False)

In [30]:
#np.mean(pred_x,pred_l,pred_cat)
input_mask_data=np.mean( np.array([ pred_x,pred_l,pred_cat ]), axis=0 )
pred_e = input_mask_data > .66
pred_e=pred_e.astype(int)

In [31]:
index=pd.DataFrame({'ID':test_id.ID})
results=pd.DataFrame(pred_e.reshape(-1,1)) 
out=index.merge(results,left_index=True,right_index=True)
out.columns=['ID','Is_Churn']
out
out.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/Ensemb1.csv',index=False)